In [1]:
import pandas as pd
import re
import nltk
from nltk.stem import RSLPStemmer

In [2]:
def orengo(song):
    st = RSLPStemmer()
    words = filter(lambda x: len(x)>0, song)
    word_list = list(map(lambda x: st.stem(x), words))
    return word_list

In [3]:
def remove_stopwords(doc):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    new_doc = list(filter(lambda x: x not in stopwords, doc))
    return new_doc

In [4]:
df = pd.read_csv('.\data\dataset_lyrics.csv')

In [5]:
#removes compilations
df = df[(df.artist != '/musicas-catolicas/')&(df.artist != '/harpa-crista/')&(df.artist != '/corinhos-evangelicos/')&
        (df.artist != '/hinos/')&(df.artist != '/colecao-amo-voce/')&(df.artist != '/umbanda/')&(df.artist != '/alok/')]

In [6]:
#data cleansing
df['lyrics'] = df['lyrics'].str.strip('<div data-plugin=""googleTranslate"" id=""lyrics"">')
df['lyrics'] = df['lyrics'].str.strip('<img alt="Instrumental" class="instrumental-icon" src="/img/etc/instrumental.png"/')
df['lyrics'] = df['lyrics'].str.replace('\n', ' ')
df['lyrics'] = df['lyrics'].str.replace('\r', '')
df['lyrics'] = df['lyrics'].apply(lambda x: re.sub('[!,.;:?()""|]', "", x))
df['lyrics'] = df['lyrics'].apply(lambda x: x.lower())

In [7]:
#creates new df where the column lyrics carries every song of a given artist
df1 = df.groupby('artist')['lyrics'].apply(' '.join).reset_index()

In [8]:
#applies stemmer and creates new column
df1['orengo'] = df1['lyrics'].apply(lambda x: orengo(x.split(' ')))

In [9]:
#removes stopwords and creates new column
df1['no_stopwords'] = df1['lyrics'].apply(lambda x: remove_stopwords(x.split(' ')))

In [10]:
#vocabulary
df1['unique'] = df1['lyrics'].apply(lambda x: len(set(x.split(' '))))

In [11]:
#vocabulary after stemming
df1['orengo_unique'] = df1['orengo'].apply(lambda x: len(set(x)))

In [12]:
#vocabulary without considering stopwords
df1['ns_unique'] = df1['no_stopwords'].apply(lambda x: len(set(x)))

In [13]:
#vocabulary without considering stopwords after stemming
df1['ns_orengo_unique'] = df1['no_stopwords'].apply(lambda x: len(set(orengo(x))))

In [14]:
df1.head()

,artist,lyrics,orengo,no_stopwords,unique,orengo_unique,ns_unique,ns_orengo_unique
0,/aline-barros/,coro bendecid al señor alma mia tu nombre sant...,"[cor, bendecid, al, señ, alm, mia, tu, nombr, ...","[coro, bendecid, al, señor, alma, mia, nombre,...",5614,3554,5469,3491
1,/amado-batista/,pare de pensar em besteira se ligue no que eu ...,"[par, de, pens, em, beste, se, lig, no, que, e...","[pare, pensar, besteira, ligue, vou, dizer, ge...",4519,2590,4379,2536
2,/ana-carolina/,110000 nega marrenta 110000 nega marrenta 1100...,"[110000, neg, marrent, 110000, neg, marrent, 1...","[110000, nega, marrenta, 110000, nega, marrent...",3504,2515,3383,2452
3,/anavitoria/,acorda meu bem amanhã já é outra história em p...,"[acord, meu, bem, amanhã, já, é, outr, histór,...","[acorda, bem, amanhã, outra, história, paz, va...",1452,1123,1357,1062
4,/anderson-freire/,senhor lembra-te de mim quando entrares no teu...,"[senh, lembra-t, de, mim, qu, entr, no, teu, r...","[senhor, lembra-te, mim, entrares, reino, , -,...",2771,1762,2646,1697
